In [ ]:
import numpy as np
import matplotlib.pyplot as plt
measure_dict_list=np.load('./measure_dict_list.npy', allow_pickle=True)

measure_matrix=[]
for measure_dict in measure_dict_list:
    row_vector=[]
    for i in range(32):
        a=str(np.base_repr(i, padding=5)[-5:])
        row_vector.append(measure_dict.get(a, 0))
    measure_matrix.append(row_vector)
measure_matrix=np.array(measure_matrix)

def IBU(p_measure, epochs=10):
    p_old=np.ones(32)/32
    #p_measure=np.ones(32)/32
    for _ in range(epochs):
        p_new=(measure_matrix@(p_measure/(p_old@measure_matrix)))*p_old
        p_old=p_new
    return p_new

data=[]
for counts_dict in counts:
    row_vector=np.zeros(32)
    for i in counts_dict.keys():
        row_vector[int(i,2)]=counts_dict[i]
    data.append(row_vector/np.sum(row_vector))
data=np.array(data)
data_IBU=np.array(list(map(IBU, data)))

for i in range(32):
    a=np.base_repr(i,2,padding=5)[-5:]
    print(a)
    
dict_list=[]
for vector_IBU in data_IBU:
    dict_a={}
    for idx, j in enumerate(vector_IBU):
        dict_a[np.base_repr(idx,2,padding=5)[-5:]]=j
    dict_list.append(dict_a)
    
from qiskit.visualization import plot_histogram
plot_histogram(dict_list, bar_labels=False, title='raw+IBU')

def counts2corr(counts):
    dict_new={}
    for key in counts.keys():
        if np.sum(np.array(list(key), dtype=np.int32))==2:
            dict_new[key]=counts[key]
    return dict_new

dict_IBU_list=list(map(counts2corr, dict_list))


counts, raw
dict_list, raw+measurement_error_mitigation
dict_IBU_list, raw+masurement_error_mitigate+symmetry

def circ(t1,t2,t3,t4,t5,t6,N):
    t_list=[t1,t2,t3,t4,t5,t6]
    block_list=[]
    for _ in range(N):
        circ_list=[]
        for t in t_list:
            pauli_sample=np.random.choice(['00','01','10','11'], size=(1, 6))
            circ_t=ramdom_sampling(t, pauli_sample)[0]
            
            #compile을 위해서 measurement와 barrier를 없앰
            circ_t.remove_final_measurements()
            circ_t = RemoveBarriers()(circ_t)
            
            circ_list.append(circ_t)
        qc=QuantumCircuit(5)
        #particle number를 2개 사용!
        qc.ry(np.pi/4,0)
        qc.ry(np.pi/3,1)
        qc.ry(np.pi/6,2)
        qc.ry(np.pi/5,3)
        qc.ry(np.pi/6,4)

        
        
        qc.compose(circ_list[0],[1,2],inplace=True)
        qc.compose(circ_list[1],[0,1],inplace=True)
        qc.compose(circ_list[2],[2,3],inplace=True)
        qc.compose(circ_list[3],[1,2],inplace=True)
        
        qc.compose(circ_list[4],[3,4],inplace=True)
        qc.compose(circ_list[5],[2,3],inplace=True)
        
        qc.measure_all()
        block_list.append(qc)
    
    return block_list

def counts2corr(counts):
    dict_new={}
    for key in counts.keys():
        if np.sum(np.array(list(key), dtype=np.int32))==2:
            dict_new[key]=counts[key]
    return dict_new


def IBU(p_measure, epochs=10):
    p_old=np.ones(32)/32
    #p_measure=np.ones(32)/32
    for _ in range(epochs):
        p_new=(measure_matrix@(p_measure/(p_old@measure_matrix)))*p_old
        p_old=p_new
    return p_new

import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
%matplotlib inline

two_pauli_list=[]
for i in ['0','1','2','3']:
    for j in ['0','1','2','3']:    
        two_pauli_list.append(i+j)

dict_pauli={}
dict_pauli['00']='00'
dict_pauli['01']='01'
dict_pauli['02']='32'
dict_pauli['03']='33'

dict_pauli['10']='11'
dict_pauli['11']='10'
dict_pauli['12']='23'
dict_pauli['13']='22'

dict_pauli['20']='21'
dict_pauli['21']='20'
dict_pauli['22']='13'
dict_pauli['23']='12'

dict_pauli['30']='30'
dict_pauli['31']='31'
dict_pauli['32']='02'
dict_pauli['33']='03'

def helper_cx(a,b,controll=0,target=1):
    #a,b : list
    qc=QuantumCircuit(2)
    if a[0]=='1':
        qc.x(0)
    elif a[0]=='2':
        qc.y(0)
    elif a[0]=='3':
        qc.z(0)
        
    if a[1]=='1':
        qc.x(1)
    elif a[1]=='2':
        qc.y(1)
    elif a[1]=='3':
        qc.z(1)
        
    qc.barrier()
    qc.cx(controll,target)
    qc.barrier()
    
    if b[0]=='1':
        qc.x(0)
    elif b[0]=='2':
        qc.y(0)
    elif b[0]=='3':
        qc.z(0)
        
    if b[1]=='1':
        qc.x(1)
    elif b[1]=='2':
        qc.y(1)
    elif b[1]=='3':
        qc.z(1)
        
    return qc

def ramdom_sampling(t, a_list):
    circ_list=[]
    for a in a_list:
        qc=QuantumCircuit(2)
        b=list(map(lambda x: dict_pauli[x], a))
        qc.compose(helper_cx(a[0], b[0]), [0,1], inplace=True)
        qc.sx(0)
        qc.rz(-np.pi/2,0)
        qc.sx(0)
        qc.x(0)
        
        qc.compose(helper_cx(a[1], b[1]), [1,0], inplace=True)

        qc.rz(-t/2,0)
        
        qc.compose(helper_cx(a[2], b[2]), [1,0], inplace=True)

        qc.rz(t/2,0)
        qc.sx(0)
        qc.rz(np.pi/2,0)
        qc.sx(0)
        qc.x(0)
        qc.compose(helper_cx(a[3], b[3]), [0,1], inplace=True)
        qc.measure_all()
        circ_list.append(qc)
    return circ_list
N=10
a_list=np.random.choice(two_pauli_list, size=(N,4))
circ_list=ramdom_sampling(np.pi, a_list)
t_qc=transpile(circ_list, basis_gates=['sx','I','cx','x','rz'], optimization_level=3)


def circ(t1,t2,t3,t4,t5,t6,N):
    t_list=[t1,t2,t3,t4,t5,t6]
    block_list=[]
    for _ in range(N):
        circ_list=[]
        for t in t_list:
            pauli_sample=np.random.choice(['00','01','10','11'], size=(1, 6))
            circ_t=ramdom_sampling(t, pauli_sample)[0]
            
            #compile을 위해서 measurement와 barrier를 없앰
            circ_t.remove_final_measurements()
            circ_t = RemoveBarriers()(circ_t)
            
            circ_list.append(circ_t)
        qc=QuantumCircuit(5)
        #particle number를 2개 사용!
        qc.x(0)
        qc.x(1)
        
        qc.compose(circ_list[0],[1,2],inplace=True)
        qc.compose(circ_list[1],[0,1],inplace=True)
        qc.compose(circ_list[2],[2,3],inplace=True)
        qc.compose(circ_list[3],[1,2],inplace=True)
        
        qc.compose(circ_list[4],[3,4],inplace=True)
        qc.compose(circ_list[5],[2,3],inplace=True)
        
        qc.measure_all()
        block_list.append(qc)
    
    return block_list

def counts2corr(counts):
    dict_new={}
    for key in counts.keys():
        if np.sum(np.array(list(key), dtype=np.int32))==2:
            dict_new[key]=counts[key]
    return dict_new